In [1]:
# In the root of the repo run "pip install --editable ."

# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

# Replace sample_func_dir by the name of directory in src/ and replace sample_func_file the
# file name in src/sample_func_dir

In [2]:
from src.data.import_data import import_df_from_zip_pkl
import rdtools
import pickle
import time

In [8]:
def CODS_with_bootstrap(synth_type, index=0, realizations=512, verbose=False):
    """
    
    """  
    # Load datasets
    path_to_zip_pkl_pi = "../data/raw/synthetic_" + synth_type + "_pi_daily.zip"
    try:
        df = import_df_from_zip_pkl(path_to_zip_pkl_pi, index=index, verbose=True, minofday=False)
    except:
        if verbose:
            print("No available synthetic dataset, the availabe types are 'basic', 'soil', 'soil_weather', 'weather'")
    
    start_time = time.time() # remove?
    
    # Initialize instance
    cods_instance = rdtools.soiling.cods_analysis(df.PI)
    # run algoritm
    cods_instance.run_bootstrap(realizations, verbose=verbose)
    
    end_time = time.time() # remove?
    print("--- %s min ---" %((end_time - start_time)/60.)) # remove?
    
    # save results
    _file = open("../data/processed/cods_results_" + synth_type + "_" + str(index) + "_" + str(realizations) + ".pkl", "wb")
    pickle.dump(cods_instance , _file)
    
    return

def load_CODS_results(synth_type, index=0, realizations=512, verbose=False):
    """
    
    """
    # Load results
    try:
        _file         = open("../data/processed/cods_results_" + synth_type + "_" + str(index) + "_" + str(realizations) + ".pkl", "rb")
        print (_file)
    except:
        if verbose:
            print("No available synthetic dataset, the availabe types are 'basic', 'soil', 'soil_weather', 'weather'")
    
    return cods_instance

In [5]:
# check everything is working correctly for 16 realizations
CODS_with_bootstrap("soil", index=0, realizations=16, verbose=True)

time for importing dataframe: 0.00 seconds
Initially fitting 16 models
# 16 | Used: 1.7 min | Left: 0.0 min | Progress: [----------------------------->] 100 %
            order    dt        pt     ff      RMSE     SR==1   weights    sss
0   [SR, SC, Rd]  0.25  0.666667   True  0.005626  0.220701  0.056888  False
1   [SR, SC, Rd]  0.25  0.666667  False  0.005315  0.243154  0.059135  False
2   [SR, SC, Rd]  0.25  1.500000   True  0.004629  0.257941  0.067098  False
3   [SR, SC, Rd]  0.25  1.500000  False  0.004629  0.257941  0.067098  False
4   [SR, SC, Rd]  0.75  0.666667   True  0.005610  0.233297  0.056470  False
5   [SR, SC, Rd]  0.75  0.666667  False  0.005315  0.243154  0.059135  False
6   [SR, SC, Rd]  0.75  1.500000   True  0.005575  0.278751  0.054807  False
7   [SR, SC, Rd]  0.75  1.500000  False  0.004629  0.257941  0.067099  False
8   [SC, SR, Rd]  0.25  0.666667   True  0.005736  0.227820  0.055474  False
9   [SC, SR, Rd]  0.25  0.666667  False  0.005629  0.280942  0.054188 

/home/maria/miniconda3/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,


# 16 | Used: 0.3 min | Left: 0.0 min | Progress: [----------------------------->] 100 %
Final RMSE: 0.00467
--- 2.0085633516311647 min ---


In [7]:
# check loading CODS results
cods_instance = load_CODS_results("soil", index=0, realizations=16, verbose=False)

<_io.BufferedReader name='../data/processed/cods_results_soil_0_16.pkl'>


In [28]:
results_df = cods_instance.result_df
results_df.head()

,soiling_ratio,soiling_rates,cleaning_events,seasonal_component,degradation_trend,total_model,residuals,SR_low,SR_high,rates_low,rates_high,bt_soiling_ratio,bt_soiling_rates,seasonal_low,seasonal_high,model_low,model_high
datetime,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00-05:00,0.963832,0.000686,0.047082,0.994163,1.000000,0.881136,0.922474,0.955449,0.980586,0.000081,0.001268,0.967095,0.000854,0.989694,0.997655,0.883970,0.898860
2015-01-02 00:00:00-05:00,0.964454,0.000559,0.017457,0.994214,0.999976,0.881728,0.920791,0.956372,0.981276,0.000076,0.001224,0.967939,0.000842,0.989686,0.997727,0.884926,0.899176
2015-01-03 00:00:00-05:00,0.964967,0.000467,0.008313,0.994267,0.999952,0.882223,0.919191,0.957191,0.982244,0.000066,0.001217,0.968772,0.000832,0.989684,0.997823,0.885765,0.899180
2015-01-04 00:00:00-05:00,0.965404,0.000408,0.010968,0.994321,0.999928,0.882650,0.917691,0.958095,0.982847,0.000059,0.001237,0.969600,0.000826,0.989694,0.997916,0.886411,0.899741
2015-01-05 00:00:00-05:00,0.965797,0.000379,0.015678,0.994379,0.999905,0.883040,0.916184,0.958939,0.984057,0.000057,0.001234,0.970421,0.000822,0.989718,0.997994,0.887198,0.900361


In [ ]:
# Load datasets
synth_type = "soil"
index      = 20
path_to_zip_pkl_pi = "../data/raw/synthetic_" + synth_type + "_pi_daily.zip"
df = import_df_from_zip_pkl(path_to_zip_pkl_pi, index=index, verbose=True, minofday=False)

cods_instance = rdtools.soiling.cods_analysis(df.PI)

cods_instance.iterative_signal_decomposition(perfect_cleaning=True, order=['SC', 'SR', 'Rd'])

time for importing dataframe: 0.00 seconds
